In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
import math

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Dataloader

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Custom Dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data.float()

    def __getitem__(self, index):
        return self.data[index].float()

    def __len__(self):
        return len(self.data)

In [4]:
def train_epoch(train_loader, model, optimizer, loss_function):
    losses = []
    # get a batch of training data from the train_loader (DataLoader obj)
    for i, data in enumerate(train_loader):
        x = data[:,:-1].to(device)
        t = data[:,-1].to(device)
        # make predictions for this batch
        y = model(x)
        # Compute the loss
        loss = loss_function(y, t)
        # zero out the gradients so that it will not accumulate through each iteration
        optimizer.zero_grad()
        # Compute the gradents with the backward call (backprop)
        loss.backward()
        # Update weight using gradient descent 
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_epoch(valid_loader, model, loss_function):
    rmses = []
    with torch.no_grad(): 
        for i, data in enumerate(valid_loader):
            x = data[:,:-1].to(device)
            t = data[:,-1].to(device)
            # Compute prediction
            y = model(x).squeeze()
            
            rmse = math.sqrt(loss_function(y,t))
            rmses.append(rmse)
            
    return np.mean(rmses)

# Define Model Architectures

In [120]:
# Define the neural network architecture
leaky_relu = nn.LeakyReLU(negative_slope=0.2)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(94, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 12)
        self.fc4 = nn.Linear(12,8)
        self.fc5 = nn.Linear(8, 1)  

    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))     
        x = self.dropout(self.relu(self.fc4(x)))   
        x = self.fc5(x)
        return x

# Create an instance of the neural network
model = Net()
model = model.to(device)
criterion = nn.MSELoss()

# Load in Data

In [42]:
df_np = np.load('processed_train_set.npy')
df_np.shape

(1710670, 95)

In [113]:
# load in np array and DRAW SAMPLE
num_samples = int(df_np.shape[0]*1)

num_rows = df_np.shape[0]
sampled_indices = np.random.choice(num_rows, size=num_samples, replace=False)

In [114]:
# convert np to tensor
df = torch.tensor(df_np[sampled_indices])

In [115]:
df.shape

torch.Size([1710670, 95])

In [116]:
# Create a sample dataset
dataset = MyDataset(df)

# Define the sizes of train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset)-train_size

In [117]:
# Split the dataset
batch_size = 32
train_data, val_data = random_split(dataset, [train_size, val_size])

# Create DataLoader objects for train, validation, and test sets
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_data, batch_size=batch_size)
# test_loader = DataLoader(test_data, batch_size=32)

# Train Model

In [118]:
def train_model(model, model_name, epochs, lr, batch_size):
    # Define the optimizer
    train_rmses = []
    valid_rmses = []
    optimizer = torch.optim.Adam(model.parameters(), lr)
#     optimizer = torch.optim.Adam(model.parameters())
    train_accs, valid_accs = [], []
    for epoch in range(epochs):

        model.train() # gradient tracking is on

        train_loss = train_epoch(train_loader, model, optimizer, criterion)

        model.eval() # we don't need gradients on to do reporting

        train_rmse = eval_epoch(train_loader, model, criterion)
        valid_rmse = eval_epoch(valid_loader, model, criterion)
        
#         train_rmses.append(train_rmse)
#         valid_rmses.append(valid_rmse)

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss:>0.4f}, Train mse: {train_rmse:>0.2f},\
              Validation Accuracy: {valid_rmse:>0.2f} \n")
    torch.save(model.state_dict(), './{}'.format(model_name))


In [ ]:
%%time
train_model(model = model, model_name = 'main_model', \
epochs = 10, lr=0.00001, batch_size = batch_size)

Epoch: 1, Train Loss: 854925.8955, Train mse: 594.25,              Validation Accuracy: 597.34 

Epoch: 2, Train Loss: 632334.5362, Train mse: 586.84,              Validation Accuracy: 590.28 

Epoch: 3, Train Loss: 620457.4213, Train mse: 580.15,              Validation Accuracy: 583.09 

Epoch: 4, Train Loss: 611028.0100, Train mse: 578.28,              Validation Accuracy: 581.33 



In [14]:
# saved_model = Net()
# saved_model.load_state_dict(torch.load('./cse151B_project'))
# saved_model

# Test Model (unfinished cuz eric poopoo code)

In [60]:
df_np_test = np.load('processed_test_set.npy')
df_np_test.shape

(320, 94)

In [63]:
test_tensor = torch.tensor(df_np_test).to(device)
test_tensor.shape

torch.Size([320, 94])

In [108]:
model = model.float()
model

Net(
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=94, out_features=48, bias=True)
  (fc2): Linear(in_features=48, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=12, bias=True)
  (fc4): Linear(in_features=12, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=1, bias=True)
)

In [109]:
pred = []
with torch.no_grad(): 
    for i in range(test_tensor.shape[0]):
        x = test_tensor[i].float().to(device)
        y = model(x)
        pred.append(y.cpu().numpy()[0])         

In [110]:
pred

[509.66367,
 126.07917,
 650.4463,
 326.46585,
 264.091,
 777.4181,
 515.1021,
 275.74536,
 132.11153,
 99.48521,
 418.01385,
 386.6475,
 830.4042,
 570.2158,
 333.39163,
 435.7697,
 329.5581,
 197.06033,
 767.50574,
 781.3656,
 492.44464,
 722.042,
 402.14267,
 326.30707,
 964.071,
 135.48036,
 422.2431,
 245.78647,
 685.8021,
 291.94916,
 328.96393,
 400.2779,
 453.23367,
 248.80649,
 289.76868,
 100.79281,
 555.4779,
 718.70074,
 369.48846,
 592.1417,
 870.1964,
 835.3273,
 332.4567,
 670.6193,
 1184.2426,
 296.03204,
 403.53625,
 475.12546,
 921.0192,
 444.74695,
 811.74536,
 991.39166,
 223.79556,
 1245.1158,
 321.01807,
 365.1096,
 136.68274,
 735.2136,
 394.27075,
 459.00232,
 495.26562,
 974.18225,
 668.09753,
 592.87744,
 358.51028,
 431.99054,
 480.58844,
 629.00024,
 248.0975,
 337.9201,
 245.9079,
 679.3573,
 101.97326,
 1023.60596,
 278.4078,
 1514.015,
 230.07062,
 1425.5544,
 299.24612,
 400.22882,
 1015.5572,
 299.90576,
 829.5895,
 763.23206,
 256.97028,
 462.01776,
 4

In [111]:
test_df = pd.read_csv('test_public.csv')
test_data = {'TRIP_ID': test_df['TRIP_ID'],
    'TRAVEL_TIME': pred}
output = pd.DataFrame(data = test_data)
output

,TRIP_ID,TRAVEL_TIME
0,T1,509.663666
1,T2,126.079170
2,T3,650.446289
3,T4,326.465851
4,T5,264.091003
...,...,...
315,T323,297.736023
316,T324,81.058113
317,T325,502.314697
318,T326,749.325562


In [112]:
output.to_csv('zeds_test_june5_3.csv', index=False)
